In [1]:
# Imports and Utility functions
import subprocess

def clear_keri():
    path = "/usr/local/var/keri/"
    confirm = input("🚨 This will clear your keystore. Are you sure? (y/n): ")
    if confirm.lower() == "y":
        print("Proceeding...")
        try:
            subprocess.run(["rm", "-rf", path], check=True)
            print(f"✅ Successfully removed: {path}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Error removing {path}: {e}")
    else:
        print("Operation cancelled.")

# Signatures

Having explored KERI Identifiers (AIDs) and their management, we now focus on digital signatures. This section explains what digital signatures are, their crucial properties, and how they operate within KERI.

## What is a Digital Signature?

A digital signature is a cryptographic mechanism used to provide assurance about the authenticity and integrity of digital data. It serves a similar purpose to a handwritten signature but offers significantly stronger guarantees through cryptography.

The process generally involves three stages:

1.  **Signing:**
    * The signer (e.g., an AID Controller in KERI) takes the information they want to sign.
    * They first create a condensed representation of the information, known as a hash.
    * Using their unique private signing key, they apply a signing algorithm to this hash. The result is the digital signature.
    * Only someone possessing the private key can generate a valid signature for that key.

2.  **Attaching:**
    * The generated signature is typically attached to the original information.

3.  **Verification:**
    * Anyone receiving the information and signature can verify its validity using the signer's corresponding public key.
    * The verifier applies a verification algorithm using the original information, the signature, and the public key.
    * This algorithm checks if the signature correctly corresponds to the information and the provided public key.
    * **Outcome:**
        * **Valid Signature:** If the algorithm accepts, the verifier has high confidence in the information's authenticity and integrity.
        * **Invalid Signature:** If the algorithm rejects, the information may have been tampered with, the signature might be corrupt, or the legitimate holder of the private key didn't generate it.

Successful verification confirms:

* **Authenticity:** The information originated from the owner of the key pair.
* **Integrity:** The information has not been altered since it was signed.

## Non-Repudiation

Digital signatures provide non-repudiation. This means the signer cannot credibly deny having signed the information once a valid signature is verified using their public key. Because generating the signature requires the private key (which should be kept secret by the owner), a valid signature serves as strong evidence of the signer's action.

## Signatures and Verification Specific to KERI

In KERI, digital signatures are fundamental for establishing trust and verifying the authenticity of Key Events and other interactions associated with an AID. They cryptographically link actions and data back to the identifier's controlling keys.

While the verification algorithm is standard, the key challenge for a Verifier in KERI is obtaining the correct public key(s) that were authoritative for the AID at the time the information was signed.

The Verifier must perform these steps:

1.  **Identify the Authoritative Public Key(s):**
    * For an AID's inception event, the initial public key(s) are derived directly from the AID Prefix itself (leveraging KERI's self-certifying nature).
    * For subsequent events (like rotations or interactions), the Verifier must consult the AID's Key Event Log. The KEL provides the history of key changes, allowing the Verifier to determine which public key(s) were valid at the specific point in time the event or message was signed.

2.  **Perform Cryptographic Verification:**
    * Once the correct public key(s) are identified, the Verifier uses them, along with the received data and signature, in the standard cryptographic verification algorithm (as described earlier).

This reliance on the KEL to track key state over time is crucial for maintaining the security of interactions with KERI identifiers long after their initial creation.

<div class="alert alert-info">
  <b>ℹ️ NOTE</b><hr>
    There's a subtle difference between a <b>Verifier</b> (who checks cryptographic correctness according to KERI rules) and a <b>Validator</b> (who might perform broader checks, including business logic, and broader trust policies in addition to verification). In KERI discussions, "Verifier" typically emphasizes the cryptographic checks.
</div>

## Practical Example: Signing and Verifying with KLI

Let's see how signing and verification work using the KLI commands.

### Setup Keystore and Identifier

First, we need a keystore and an identifier.

In [25]:
keystore_name="signature-key-store"
keystore_passcode="xSLg286d4iWiRg2mzGYca"
salt="0ABeuT2dErMrqFE5Dmrnc2Bq"
aid_alias = "aid-sign55555"

!kli init --name {keystore_name} --passcode {keystore_passcode} --salt {salt}
!kli incept --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --icount 1 --isith 1 --ncount 0 --nsith 0 --toad 0

KERI Keystore created at: /usr/local/var/keri/ks/signature-key-store
KERI Database created at: /usr/local/var/keri/db/signature-key-store
KERI Credential Store created at: /usr/local/var/keri/reg/signature-key-store
	aeid: BD-1udeJaXFzKbSUFb6nhmndaLlMj-pdlNvNoN562h3z
ERR: Already incepted pre=BLj9982ny88YokEFdjiyUKAQ4Gavy0EMdNBLQmllGfDA.


### Sign a Message 

Now, let's sign a simple text message `"hello world"` using the private key associated with our `aid-sign` identifier.

In [26]:
!kli sign --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} --text "hello world"

1. AAA5bWbt5D50dDU-mO7_7S3xKCRfXS9eRublKicsNiV--s5O9ZH2b3Gfols77RMzmAjEP0CLS_Vyoo23aB8PlLMA


The output is the digital signature generated for the text "hello world" using the private key of the AID  

### Verify the Signature 

We can now use the kli verify command to check if the signature is valid for the given message and identifier (prefix).

In [27]:
!kli verify --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --prefix BPkrtxL07k1cQhoRlF2illnFZJc_Veywvg4xvLFPtCE3 \
    --text "hello world" \
    --signature AAAzpKaOVx-4G1cfldpRx-mHi1UGS-smitfVCSM47v_mECPkgHVPfqOogqseh8GwPyua0JFi3wNPRHEnowxgDYUE


Signature 1 is valid.


The command confirms the signature is valid. It used the public key associated with the prefix  (which KLI retrieves from the keystore in this case) to verify the signature against the provided text.

### Tampered Signature

What happens if the signature is altered even slightly? Let's change the last character of the signature and try to verify again.


In [28]:
!kli verify --name {keystore_name} --alias {aid_alias} --passcode {keystore_passcode} \
    --prefix BPkrtxL07k1cQhoRlF2illnFZJc_Veywvg4xvLFPtCE3 \
    --text "hello world" \
    --signature AAAzpKaOVx-4G1cfldpRx-mHi1UGS-smitfVCSM47v_mECPkgHVPfqOogqseh8GwPyua0JFi3wNPRHEnowxgDYUU # Tampered last character

ERR: Signature 1 is invalid.


As expected, the verification fails. Even a tiny change invalidates the signature, demonstrating the integrity protection it provides.